In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/data-analyst-jobs/DataAnalyst.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

### Clean the data

In [ ]:
# remove the unnamed:0 column in the dataset

df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
# Missing data

df.isnull().sum()

One company name is missing out of 2254 entries

In [ ]:
df['Job Title'].value_counts()

In [ ]:
df['Salary Estimate'].value_counts()

In [ ]:
df['Rating'].value_counts()

In [ ]:
df['Founded'].value_counts()

In [ ]:
df['Location'].value_counts()

In [ ]:
df['Size'].value_counts()

In [ ]:
df['Headquarters'].value_counts()

In [ ]:
df['Type of ownership'].value_counts()

In [ ]:
df['Industry'].value_counts()

In [ ]:
df['Sector'].value_counts()

In [ ]:
df['Revenue'].value_counts()

In [ ]:
df['Competitors'].value_counts()

In [ ]:
df['Easy Apply'].value_counts()

##### We can observe that almost all columns contain -1 as a value which can be replaced with Nan
##### The -1 is having int, categorical(string) and float datatype and each should be done seperately

In [ ]:
df = df.replace(-1,np.nan)
df = df.replace(-1.0,np.nan)
df = df.replace('-1',np.nan)

In [ ]:
df.isnull().sum()

##### columns easy apply and competitors have more than 50 percent data as null values

In [ ]:
# we can observe from the dataset that the company name is having \n rating at the end of the name annd it can be removed

df['Company Name'],_ = df['Company Name'].str.split('\n', 1). str

In [ ]:
# we can observe that the column job title has both title and department in same column and they can be seperated

df['Job Title'], df['Department'] = df['Job Title'].str.split(',', 1).str

In [ ]:
# We can observe from the salary estimate colmn that the salary is estimated by glassdoor at the end of estimated values and this can be removed.

df['Salary Estimate'],_ = df['Salary Estimate'].str.split('(', 1).str

In [ ]:
df.head()

In [ ]:
# In salary estimate we can observe the presence of $ and K at left and right of the value respectively and they can be removed

df['Minimum Salary Estimate'], df['Maximum Salary Estimate'] = df['Salary Estimate'].str.split('-').str

df['Minimum Salary Estimate'] = df['Minimum Salary Estimate'].str.strip(' ').str.lstrip('$').str.rstrip('K').fillna(0).astype('int')

df['Maximum Salary Estimate'] = df['Maximum Salary Estimate'].str.strip(' ').str.lstrip('$').str.rstrip('K').fillna(0).astype('int')

In [ ]:
df.head()

In [ ]:
# now we can drop the salary estimate column

df.drop(['Salary Estimate'], axis = 1, inplace = True)

In [ ]:
# As we can see easy apply column has Nan values hey can be converted to bool False

df['Easy Apply'] = df['Easy Apply'].fillna(False).astype(bool)

In [ ]:
df.head()

In [ ]:
# True in easy apply represents that the company is hiring at present
# we can now plot the company wise hiring 

df_hiring = df[df['Easy Apply'] == True]
df_company = df_hiring.groupby('Company Name').count().reset_index()
df_company = df_company.sort_values('Easy Apply', ascending = True)
df_company

In [ ]:
# We can now visualize the jobs company wise
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize = (18, 7))
sns.barplot('Company Name', 'Easy Apply', data = df_company, )
plt.xticks(rotation = 90)
plt.show()

In [ ]:
# We can visulaize location wise salary trends 
df_location = df.groupby('Location')[['Minimum Salary Estimate', 'Maximum Salary Estimate']].mean().sort_values(['Maximum Salary Estimate', 'Minimum Salary Estimate'],ascending = False).head(50)
df_location

In [ ]:
df_location.plot.bar(stacked = True, figsize = (20,6))
plt.show()

In [ ]:
df.isnull().sum()

In [ ]:
# We can replace missing values in rating with mean value
df['Rating'].fillna(df['Rating'].mean(), inplace = True)

In [ ]:
df.isnull().sum()

In [ ]:
df.fillna('Unknown', inplace = True)

In [ ]:
df_rating = df.groupby('Rating')['Company Name'].count()
df_rating = pd.DataFrame(df_rating).rename(columns = {'Company Name': 'Count of companies'}).reset_index()
df_rating = df_rating.sort_values('Count of companies', ascending = False)
df_rating

In [ ]:
plt.figure(figsize = (20,6))
sns.barplot(x = df_rating['Rating'],y = df_rating['Count of companies'])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
# Top places where most companies headquarters are located

df_hq = df.groupby('Headquarters')['Company Name'].count()
df_hq = pd.DataFrame(df_hq).rename(columns = {'Company Name': 'Count of companies'}).reset_index()
df_hq = df_hq.sort_values('Count of companies', ascending = False).head(30)
df_hq

In [ ]:
plt.figure(figsize = (10,6))
sns.barplot(x = df_hq['Headquarters'],y = df_hq['Count of companies'])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
df['Size']

In [ ]:
df_size = df.groupby('Size')['Company Name'].count()
df_size = pd.DataFrame(df_size).rename(columns = {'Company Name': 'Count of companies'}).reset_index()
df_size = df_size.sort_values('Count of companies', ascending = False)
df_size

In [ ]:
plt.figure(figsize = (10,6))
sns.barplot(x = df_size['Size'],y = df_size['Count of companies'])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
df_found = df.groupby('Founded')['Company Name'].count()
df_found = pd.DataFrame(df_found).rename(columns = {'Company Name': 'Count of companies'}).reset_index()
df_found = df_found.sort_values('Count of companies', ascending = False).head(50)
df_found

In [ ]:
plt.figure(figsize = (15,5))
sns.barplot(x = df_found['Founded'],y = df_found['Count of companies'])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
df_owner = df.groupby('Type of ownership')['Company Name'].count()
df_owner = pd.DataFrame(df_owner).rename(columns = {'Company Name': 'Count of companies'}).reset_index()
df_owner = df_owner.sort_values('Count of companies', ascending = False)


plt.figure(figsize = (10,5))
sns.barplot(x = df_owner['Type of ownership'],y = df_owner['Count of companies'])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
df_industry = df.groupby('Industry')['Company Name'].count()
df_industry = pd.DataFrame(df_industry).rename(columns = {'Company Name': 'Count of companies'}).reset_index()
df_industry = df_industry.sort_values('Count of companies', ascending = False)


plt.figure(figsize = (20,5))
sns.barplot(x = df_industry['Industry'],y = df_industry['Count of companies'])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
df_sector = df.groupby('Sector')['Company Name'].count()
df_sector = pd.DataFrame(df_sector).rename(columns = {'Company Name': 'Count of companies'}).reset_index()
df_sector = df_sector.sort_values('Count of companies', ascending = False)


plt.figure(figsize = (20,5))
sns.barplot(x = df_sector['Sector'],y = df_sector['Count of companies'])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
df_revenue = df.groupby('Revenue')['Company Name'].count()
df_revenue = pd.DataFrame(df_revenue).rename(columns = {'Company Name': 'Count of companies'}).reset_index()
df_revenue = df_revenue.sort_values('Count of companies', ascending = False)


plt.figure(figsize = (20,5))
sns.barplot(x = df_revenue['Revenue'],y = df_revenue['Count of companies'])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
df_ind = df.groupby('Industry')[['Minimum Salary Estimate', 'Maximum Salary Estimate']].mean().sort_values(['Maximum Salary Estimate', 'Minimum Salary Estimate'],ascending = False)
df_ind = df_ind.rename(columns = {'Minimum Salary Estimate' : 'Average Minimum Salary', 'Maximum Salary Estimate' : 'Average Maximum Salary'})
df_ind

In [ ]:
df_ind.plot.bar(stacked = True, figsize = (25,5))
plt.show()

In [ ]:
df_open = pd.DataFrame(df[df['Easy Apply'] == True]['Job Title'].value_counts()).rename(columns={'Job Title':'Number of openings'}).reset_index().rename(columns={'index':'Job Title'})
df_open

In [ ]:
plt.figure(figsize = (25, 5))
sns.barplot(df_open['Job Title'], df_open['Number of openings'])
plt.xticks(rotation = 90)
plt.show()

## ***Please upvote if you like the content***